In [1]:
# work in C:\Users\mingzhou
import os
from _datetime import datetime
from getpass import getpass

import pandas as pd
from arcgis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features.elevation import summarize_elevation

In [2]:
gis = GIS("pro")

ImportError: Could not import arcpy

In [46]:
ELEVATION_DATA_DIR = "elevationData"

In [50]:
df = pd.read_csv(os.path.join(ELEVATION_DATA_DIR, "nodes.csv"),header=0,index_col = 0)

In [51]:
df.head()

,latitude,longitude
33295049,44.925143,-93.265072
33295062,44.980513,-93.358550
33295067,44.980893,-93.358120
33295073,44.925143,-93.266334
33295086,44.923332,-93.265099


In [54]:
df = df.sort_values(['longitude','latitude'])

In [55]:
df.head()

,latitude,longitude
1832746163,44.792514,-93.497490
540421540,44.974351,-93.497438
538452673,44.974630,-93.497399
1139357132,44.780499,-93.497372
188040301,44.772367,-93.497348


In [56]:
sdf = pd.DataFrame.spatial.from_xy(df, "longitude", "latitude")
sdf_parts = [
    sdf[start_index : start_index + 1000].copy()
    for start_index in range(0, len(sdf), 1000)
]
print(f"{len(sdf)} records, {len(sdf_parts)} API calls required")

26871 records, 27 API calls required


In [57]:
sdf.head()

,latitude,longitude,SHAPE
1832746163,44.792514,-93.497490,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
540421540,44.974351,-93.497438,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
538452673,44.974630,-93.497399,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
1139357132,44.780499,-93.497372,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."
188040301,44.772367,-93.497348,"{""spatialReference"": {""wkid"": 4326}, ""x"": -93...."


In [58]:
for i, logrecord_part in enumerate(sdf_parts, 1):
    print(
        f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} start part {i:3d}/{len(sdf_parts):,} with {len(logrecord_part)} records"
    )

    result = summarize_elevation(
        input_features=logrecord_part,
        dem_resolution="10m",
    )

    rdf = result.sdf
    rdf = rdf.drop(
        columns=[
            "OBJECTID",
            "ID",
            "Source",
            "Source_URL",
            "SHAPE",
        ]
    )

    file_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
    rdf.to_csv(os.path.join(ELEVATION_DATA_DIR, f"logrecord_with_elevation_{file_timestamp}.csv"))

2021-05-18 13:05:09 start part   1/27 with 1000 records
2021-05-18 13:06:52 start part   2/27 with 1000 records
2021-05-18 13:08:38 start part   3/27 with 1000 records
2021-05-18 13:10:27 start part   4/27 with 1000 records
2021-05-18 13:12:12 start part   5/27 with 1000 records
2021-05-18 13:13:58 start part   6/27 with 1000 records
2021-05-18 13:15:44 start part   7/27 with 1000 records
2021-05-18 13:17:28 start part   8/27 with 1000 records
2021-05-18 13:19:12 start part   9/27 with 1000 records
2021-05-18 13:20:56 start part  10/27 with 1000 records
2021-05-18 13:22:44 start part  11/27 with 1000 records
2021-05-18 13:24:27 start part  12/27 with 1000 records
2021-05-18 13:26:13 start part  13/27 with 1000 records
2021-05-18 13:27:56 start part  14/27 with 1000 records
2021-05-18 13:29:40 start part  15/27 with 1000 records
2021-05-18 13:31:25 start part  16/27 with 1000 records
2021-05-18 13:33:08 start part  17/27 with 1000 records
2021-05-18 13:34:52 start part  18/27 with 1000 

In [65]:
for f,_,n in os.walk('elevationData/dataWithElevation'):
    for i, file in enumerate(n):
        fileName = os.path.join(f,file)
        if i == 0:
            dfWithElevation = pd.read_csv(fileName)
        else:
            df = pd.read_csv(fileName)
            dfWithElevation = pd.concat([dfWithElevation,df])


In [66]:
dfWithElevation

,Unnamed: 0,latitude,longitude,MinElevation,MeanElevation,MaxElevation,DEMResolution,ProductName
0,0,44.792514,-93.497490,228.30063,228.30063,228.30063,10m,NED_1r3_arcsec
1,1,44.974351,-93.497438,293.83191,293.83191,293.83191,10m,NED_1r3_arcsec
2,2,44.974630,-93.497399,295.07681,295.07681,295.07681,10m,NED_1r3_arcsec
3,3,44.780499,-93.497372,243.75601,243.75601,243.75601,10m,NED_1r3_arcsec
4,4,44.772367,-93.497348,256.45248,256.45248,256.45248,10m,NED_1r3_arcsec
...,...,...,...,...,...,...,...,...
866,866,44.787572,-93.185197,286.35834,286.35834,286.35834,10m,NED_1r3_arcsec
867,867,44.891459,-93.185118,248.32486,248.32486,248.32486,10m,NED_1r3_arcsec
868,868,44.822938,-93.185047,260.46021,260.46021,260.46021,10m,NED_1r3_arcsec
869,869,44.891534,-93.185036,248.56485,248.56485,248.56485,10m,NED_1r3_arcsec


In [67]:
dfWithElevation = dfWithElevation.sort_values(['longitude','latitude'])

In [68]:
dfWithElevation.head()

,Unnamed: 0,latitude,longitude,MinElevation,MeanElevation,MaxElevation,DEMResolution,ProductName
0,0,44.792514,-93.497490,228.30063,228.30063,228.30063,10m,NED_1r3_arcsec
1,1,44.974351,-93.497438,293.83191,293.83191,293.83191,10m,NED_1r3_arcsec
2,2,44.974630,-93.497399,295.07681,295.07681,295.07681,10m,NED_1r3_arcsec
3,3,44.780499,-93.497372,243.75601,243.75601,243.75601,10m,NED_1r3_arcsec
4,4,44.772367,-93.497348,256.45248,256.45248,256.45248,10m,NED_1r3_arcsec


In [69]:
dfNoElevation = pd.read_csv(os.path.join(ELEVATION_DATA_DIR, "nodes.csv"),header=0,index_col = 0)

In [70]:
dfNoElevation = dfNoElevation.sort_values(['longitude','latitude'])

In [71]:
dfNoElevation.head()

,latitude,longitude
1832746163,44.792514,-93.497490
540421540,44.974351,-93.497438
538452673,44.974630,-93.497399
1139357132,44.780499,-93.497372
188040301,44.772367,-93.497348


In [74]:
len(dfNoElevation) == len(dfWithElevation)

True

In [76]:
dfNoElevation.index

Int64Index([1832746163,  540421540,  538452673, 1139357132,  188040301,
             188038571,  188053460,  469234455,  188048752,   34381089,
            ...
             186754632,  187858111,  186732447,  187912184,  186729966,
             186815137,   34573724,  186808023,  322935109,  187896589],
           dtype='int64', length=26871)

In [77]:
dfWithElevation.index = dfNoElevation.index

In [78]:
dfWithElevation.head()

,Unnamed: 0,latitude,longitude,MinElevation,MeanElevation,MaxElevation,DEMResolution,ProductName
1832746163,0,44.792514,-93.497490,228.30063,228.30063,228.30063,10m,NED_1r3_arcsec
540421540,1,44.974351,-93.497438,293.83191,293.83191,293.83191,10m,NED_1r3_arcsec
538452673,2,44.974630,-93.497399,295.07681,295.07681,295.07681,10m,NED_1r3_arcsec
1139357132,3,44.780499,-93.497372,243.75601,243.75601,243.75601,10m,NED_1r3_arcsec
188040301,4,44.772367,-93.497348,256.45248,256.45248,256.45248,10m,NED_1r3_arcsec


In [79]:
dfWithElevation.to_csv("nodesWithElevation.csv", columns=['latitude','longitude','MeanElevation'])